In [1]:
import os
import numpy as np
import scanpy as sc

from arboreto.algo import grnboost2, genie3
from arboreto.utils import load_tf_names
from distributed import Client, LocalCluster
from dask_jobqueue import SLURMCluster
import os

In [26]:
# read in expression matrix 
# set a working directory
wdir = "/lustre/groups/ml01/workspace/samantha.bening/Bachelor/"
os.chdir( wdir )

adata = sc.read_h5ad('data2/veo_ibd_balanced.h5ad')
adata

AnnData object with n_obs × n_vars = 176789 × 28162
    obs: 'sampID', 'LMUSampID', 'PatientID', 'AnatomicLocation', 'PoolID', 'Protocol', 'CellsLoaded', 'FreezeProtocol', 'AnatomicLocation_Relation', 'AgeYM', 'Sex', 'Inflammation', 'Viability', 'SampleDate', 'sample_LMU_info_Timepoint', 'Ischemia_time', 'n_counts', 'log_counts', 'n_genes', 'mt_frac', 'ribo_frac', 'hb_frac', 'celltype_l3', 'celltype_l1', 'celltype_l2', 'PatientID_genotype'
    uns: 'PatientID_colors', 'PatientID_genotype_colors', 'celltype_l1_colors', 'celltype_l2_colors', 'celltype_l3_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'unintegrated_X_umap'
    varm: 'PCs'
    layers: 'raw'
    obsp: 'connectivities', 'distances'

In [21]:
adata.obs['celltype_l1'].value_counts()
cell_type = "Stroma"

In [27]:
# subset for a very broad cell type
adata = adata[adata.obs['celltype_l1'] == cell_type]
# filter for genes not expressed in e.g. 30 or more cells
sc.pp.filter_genes(adata, min_cells=40)
# randomly sample 5k cells from this subset (compute limit)
a = np.zeros(adata.obs['celltype_l1'].value_counts()[cell_type], dtype=int)
a[:5000] = 1
np.random.shuffle(a)
a = a.astype(bool)
adata = adata[a, :]
adata

/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number


View of AnnData object with n_obs × n_vars = 5000 × 19330
    obs: 'sampID', 'LMUSampID', 'PatientID', 'AnatomicLocation', 'PoolID', 'Protocol', 'CellsLoaded', 'FreezeProtocol', 'AnatomicLocation_Relation', 'AgeYM', 'Sex', 'Inflammation', 'Viability', 'SampleDate', 'sample_LMU_info_Timepoint', 'Ischemia_time', 'n_counts', 'log_counts', 'n_genes', 'mt_frac', 'ribo_frac', 'hb_frac', 'celltype_l3', 'celltype_l1', 'celltype_l2', 'PatientID_genotype'
    var: 'n_cells'
    uns: 'PatientID_colors', 'PatientID_genotype_colors', 'celltype_l1_colors', 'celltype_l2_colors', 'celltype_l3_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'unintegrated_X_umap'
    varm: 'PCs'
    layers: 'raw'
    obsp: 'connectivities', 'distances'

In [4]:
# make expression matrix 
ex_matrix = adata.to_df()
ex_matrix

,AL627309.1,AL627309.3,AL669831.5,FAM87B,LINC00115,FAM41C,AL645608.2,LINC02593,SAMD11,NOC2L,...,MAFIP,AC011043.1,AL592183.1,AC007325.4,AC007325.2,AL354822.1,AC004556.1,AC233755.2,AC233755.1,AC240274.1
CID006480-1:ACCTGTCGTGGTCCCA,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.088189,...,0.0,0.0,0.088189,0.0,0.0,0.0,0.0,0.0,0.0,0.088189
CID006480-1:GTAGTACTCGAGCCTG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.149761,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID006480-1:GTGTGATGTGAGTTTC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.144580,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID006480-1:TTGGATGTCATCGTAG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.098530,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID006480-1:ACACCAACAAATCGGG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.431547,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CID008847-1:TCGCTCATCATTGCGA,0.0,0.0,0.0,0.0,0.316601,0.0,0.0,0.0,0.0,0.316601,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID008847-1:TTCAGGATCGTTCCTG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID008847-1:CCGTTCAAGCTAGCCC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
CID008847-1:GTCCTCACAAGCACAG,0.0,0.0,0.0,0.0,0.560699,0.0,0.0,0.0,0.0,0.560699,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [5]:
# make 'tf_list' which in this case is just the genes themselves (for gene-gene adjacencies)

tf_names = load_tf_names("../data/allTFs_human.txt")
len(tf_names)

1892

##  Create custom dask client

In [5]:
local_cluster = LocalCluster(n_workers=29, 
                             threads_per_worker=10) # put in the number of cores you gave the job

custom_client = Client(local_cluster)
custom_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 29
Total threads: 290,Total memory: 300.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33683,Workers: 29
Dashboard: http://127.0.0.1:8787/status,Total threads: 290
Started: Just now,Total memory: 300.00 GiB
Comm: tcp://127.0.0.1:46333,Total threads: 10
Dashboard: http://127.0.0.1:35211/status,Memory: 10.34 GiB
Nanny: tcp://127.0.0.1:42311,


## SLURM Cluster

In [14]:
client.shutdown()

cluster = SLURMCluster(
    cores=20,
    processes=4,
    memory="100GB",
    shebang='#!/bin/bash',
    queue="cpu_p",
    walltime="00:30:00",
    job_extra_directives=[
        "--qos=cpu_normal", 
        "--nice=10000",
        "-o grnboost_%j.log",
        "-e grnboost_%j.err",
    ], 
    local_directory='/lustre/groups/ml01/workspace/samantha.bening/Bachelor/src/scripts/',
    death_timeout="15s",
    interface="ib0",)

print(cluster.job_script())

#!/bin/bash

#SBATCH -J dask-worker
#SBATCH -p cpu_p
#SBATCH -n 1
#SBATCH --cpus-per-task=20
#SBATCH --mem=94G
#SBATCH -t 00:30:00
#SBATCH --qos=cpu_normal
#SBATCH --nice=10000
#SBATCH -o grnboost_%j.log
#SBATCH -e grnboost_%j.err

/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/bin/python -m distributed.cli.dask_worker tcp://10.11.1.103:43649 --name dummy-name --nthreads 5 --memory-limit 23.28GiB --nworkers 4 --nanny --death-timeout 15s --local-directory /lustre/groups/ml01/workspace/samantha.bening/Bachelor/src/scripts/ --interface ib0



In [15]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.11.1.103:8787/status,
Dashboard: http://10.11.1.103:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.11.1.103:43649,Workers: 0
Dashboard: http://10.11.1.103:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [16]:
cluster.scale(4) # scale to 8 _workers_ i.e. 4 SLURM jobs

In [18]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.11.1.103:8787/status,
Dashboard: http://10.11.1.103:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.11.1.103:43649,Workers: 0
Dashboard: http://10.11.1.103:8787/status,Total threads: 0
Started: 1 minute ago,Total memory: 0 B


## Run GRNBoost2

In [19]:
%%time
network = grnboost2(expression_data=ex_matrix,
                    tf_names=tf_names,
                    client_or_address=client)

2024-04-03 18:53:42,272 - distributed.core - ERROR - Exception while handling op scatter
Traceback (most recent call last):
  File "/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/distributed/core.py", line 970, in _handle_comm
    result = await result
             ^^^^^^^^^^^^
  File "/home/icb/samantha.bening/tools/apps/mamba/envs/grnboost/lib/python3.12/site-packages/distributed/scheduler.py", line 6094, in scatter
    raise TimeoutError("No valid workers found")
TimeoutError: No valid workers found


TimeoutError: No valid workers found

In [ ]:
# filter for only importance >= 0.001 
network = network[network['importance'] >= 0.001]

network.to_csv(f'src/SCENICfiles/adj_full_TFs.csv',  header=False, index=False)

In [8]:
custom_client.shutdown()
local_cluster.close()
